# Design and Develop the Prompt Generation System
- Users can input a description of their objective or task and specify a few scenarios along with their expected outputs. 
- Write or adopt sophisticated algorithms, you generate multiple prompt options based on the provided information. 
- This automated prompt generation process saves time and provides a diverse <br/>
range of alternatives to consider. But add an evaluation metrics that check <br/> 
whether the generated prompt candidate aligns with the input description.


### Import Packages

In [1]:
import os
import sys
from dotenv import load_dotenv,find_dotenv
sys.path.append(os.path.abspath(os.path.join('../utility')))

In [5]:
from rag_utils import data_loader, create_langchain_pipeline, create_retriever

### Load OPENAI KEY from .env

In [6]:
# Load OpenAI API key from .env file
load_dotenv(find_dotenv())

True

### Load the dataset and create chunks

In [7]:
chunks = data_loader()

2024-01-19 19:58:20,619:logger:data loaded to vector database successfully


### Create vector database and retriever

In [9]:
retriever = create_retriever(chunks)

Started /home/babi/.cache/weaviate-embedded: process ID 105855


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-01-19T19:58:34+03:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-01-19T19:58:34+03:00"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-01-19T19:58:34+03:00"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-01-19T19:58:34+03:00"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2024-01-19T19:58:34+03:00"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://12

In [10]:
retriever

In [11]:
# Specify the path to your .txt file
file_path = '../prompts/prompt-generation-prompt.txt'

# Open the file in read mode
with open(file_path, 'r') as file:
    # Read the contents of the file
    file_contents = file.read()

In [12]:
rag_chain = create_langchain_pipeline(retriever, file_contents)

/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
2024-01-19 19:58:46,019:logger:An unexpected error occurred: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'NoneType'>


In [13]:
rag_chain

In [14]:
generated_prompts = rag_chain.invoke("questions about the challenge documnet")

AttributeError: 'NoneType' object has no attribute 'invoke'

In [13]:
generated_prompts


'[\n  {\n    "prompt": "When did OpenAI showcase the capabilities of reinforcement learning algorithms through its \'OpenAI Five\' project?",\n    "ground_truth": "OpenAI showcased the capabilities of these reinforcement learning algorithms through its ‘OpenAI Five’ project in 2018."\n  },\n  {\n    "prompt": "Who founded OpenAI?",\n    "ground_truth": "OpenAI was initially founded in 2015 by Sam Altman, Elon Musk, Ilya Sutskever and Greg Brockman."\n  },\n  {\n    "prompt": "What is the goal of OpenAI?",\n    "ground_truth": "The stated goal of OpenAI is to \'advance digital intelligence in the way that is most likely to benefit humanity as a whole.\'"\n  },\n  {\n    "prompt": "What did OpenAI release in 2016?",\n    "ground_truth": "OpenAI released \'OpenAI Gym\' in 2016, a toolkit for developing and comparing reinforcement learning algorithms."\n  },\n  {\n    "prompt": "What did OpenAI achieve in the early years?",\n    "ground_truth": "OpenAI made significant progress in research

In [15]:
import json
prompt_data = json.loads(generated_prompts)
prompt_data

[{'prompt': "When did OpenAI showcase the capabilities of reinforcement learning algorithms through its 'OpenAI Five' project?",
  'ground_truth': 'OpenAI showcased the capabilities of these reinforcement learning algorithms through its ‘OpenAI Five’ project in 2018.'},
 {'prompt': 'Who founded OpenAI?',
  'ground_truth': 'OpenAI was initially founded in 2015 by Sam Altman, Elon Musk, Ilya Sutskever and Greg Brockman.'},
 {'prompt': 'What is the goal of OpenAI?',
  'ground_truth': "The stated goal of OpenAI is to 'advance digital intelligence in the way that is most likely to benefit humanity as a whole.'"},
 {'prompt': 'What did OpenAI release in 2016?',
  'ground_truth': "OpenAI released 'OpenAI Gym' in 2016, a toolkit for developing and comparing reinforcement learning algorithms."},
 {'prompt': 'What did OpenAI achieve in the early years?',
  'ground_truth': 'OpenAI made significant progress in research in deep learning and reinforcement learning in the early years.'}]

{"level":"info","msg":"Created shard langchain_249710fa43a04b028c6088aa0bcb5637_5JcTAjbnLlS2 in 2.811321ms","time":"2024-01-18T23:23:19+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-01-18T23:23:19+03:00","took":463004}
{"level":"info","msg":"Created shard langchain_4b51e0a56e34429294183d836886bbf5_chZtzv3vlG0X in 1.083476ms","time":"2024-01-18T23:25:50+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-01-18T23:25:50+03:00","took":164332}
{"level":"info","msg":"Created shard langchain_dfad81fac8e04bf9a5dc95399e7c1293_mUFMm1hng3t7 in 969.508µs","time":"2024-01-18T23:31:55+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-01-18T23:31:55+03:00","took":99800}
